# Load CSVs to Postgres

Workflow: 6 

Goal: Load all source files, TMDB, Reelgood, IMDB to database.

Result: The 3 database _Load tables are populated.

In [1]:
import os, sys
import time
import pandas as PD

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
os.chdir(os.getenv('PWD'))
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")
import django
django.setup()

'app_proj.settings'

In [4]:
import app_proj.settings as ST
import movies.models.models as MD

## Read & Insert CSVs

In [5]:
MOVIEDB_FILE = os.path.join(ST.BASE_DIR, 'movies/data/moviedb_movie.csv')
REELGOOD_FILE = os.path.join(ST.BASE_DIR, 'movies/data/reelgood_movie.csv')
IMDB_FILE = os.path.join(ST.BASE_DIR, 'movies/data/imdb_movie.csv')

In [6]:
MD.Editor.DeleteTable('All')

In [7]:
# the movie DB

moviedb_ls = MD.Editor.CSVtoDict(MOVIEDB_FILE)
len(moviedb_ls)

17668

In [8]:
#moviedb_ls[12]

In [9]:
MD.Editor.InsertDictToTable(moviedb_ls, 'MovieDB_Load')
MD.MovieDB_Load.objects.all().count()

17668

In [10]:
# reelgood

reelgood_ls = MD.Editor.CSVtoDict(REELGOOD_FILE)
len(reelgood_ls)

18498

In [11]:
#reelgood_ls[90]

In [12]:
MD.Editor.InsertDictToTable(reelgood_ls, 'Reelgood_Load')
MD.Reelgood_Load.objects.all().count()

18484

In [13]:
# imdb

imdb_ls = MD.Editor.CSVtoDict(IMDB_FILE)
len(imdb_ls)

9955

In [14]:
#imdb_ls[0]

In [15]:
MD.Editor.InsertDictToTable(imdb_ls, 'IMDB_Load')
MD.IMDB_Load.objects.all().count()

9955

## Create Master Movie Table

In [16]:
master_ls = MD.Editor.RunMasterMovies()
MD.Editor.DeleteTable('MasterMovie')
MD.Editor.InsertDictToTable(master_ls, 'MasterMovie')
MD.MasterMovie.objects.all().count()

17668

In [17]:
#master_df = PD.DataFrame(master_ls)
#master_df.head()
#master_df.info()

In [18]:
history_ls = MD.Reporter.GetDataHistory()
history_df = PD.DataFrame(history_ls).set_index('Column')
history_df

,TMDB,IMDB,Reelgood,Union-All
Column,,,,
Title,100.0,100.0,100.0,100.0
Year,100.0,100.0,100.0,100.0
Rating,0.0,40.9,43.2,46.4
Companies,65.6,92.3,0.0,81.7
Country,76.8,99.5,38.7,99.9
Language,100.0,98.7,0.0,100.0
RunTime,93.4,94.8,97.7,100.0
Crew*,94.6,99.4,0.0,53.4
Cast*,90.6,93.3,0.0,45.6


In [19]:
# for development, return the merged dataframe

df = MD.Editor.RunMaster Movies()
df[150:152]

SyntaxError: invalid syntax (<ipython-input-19-4a2bd81442f3>, line 3)

In [ ]:
df.info()

In [ ]:
df.loc[ df['Companies_tmdb'].str.contains(', The')==True , [ 'Companies_tmdb', 'Companies_imdb']][40:80]

In [ ]:
df['Rating_rlgd'].value_counts()

In [ ]:
tmdb_ genre = []
rlgd_genre = []
imdb_genre = []
for idx, row in df.iterrows():
    tmdb = row['Genres_tmdb'] or ''
    rlgd = row['Genres_rlgd'] or ''
    imdb = row['Genres_imdb'] or ''
    
    tmdb_genre += tmdb.split(', ') if type(tmdb)==str else []
    rlgd_genre += rlgd.split(', ') if type(rlgd)==str else []
    imdb_genre += imdb.split(', ') if type(imdb)==str else []
    
    tmdb_ct = PD.Series(tmdb_genre).value_counts()
    rlgd_ct = PD.Series(rlgd_genre).value_counts()
    imdb_ct = PD.Series(imdb_genre).value_counts()
    
#tmdb_ct

In [ ]:
df.loc[[111, 136, 148, 269, 272, 19, 44, 47], ['Title_tmdb', 'Budget_tmdb', 'Budget_imdb']]

In [ ]:
round( ((6.8 * 293) + (5.8 * 6457))/(293+6457) , 2)

df.dropna(subset=['Votes_imdb'])['Votes_tmdb'].astype('int').sum()
df.dropna(subset=['Votes_imdb'])['Votes_imdb'].astype('int').sum()

In [ ]:
MD.Editor.GetMasterMovie(df.iloc[56])

In [ ]:
MD.Editor.GetMasterMovie(df.iloc[166])